In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import torch

import warnings
warnings.filterwarnings("ignore") 

import sys
sys.path.append('../src')

In [3]:
# Names of the columns that are discrete
discrete_columns = [
    'data_channel_is_lifestyle',
    'data_channel_is_entertainment',
    'data_channel_is_bus',
    'data_channel_is_socmed',
    'data_channel_is_tech',
    'data_channel_is_world',
    'weekday_is_monday',
    'weekday_is_tuesday',
    'weekday_is_wednesday',
    'weekday_is_thursday',
    'weekday_is_friday',
    'weekday_is_saturday',
    'weekday_is_sunday',
    'is_weekend'
]

In [4]:
df = pd.read_csv('../data/news/news_train.csv')
df.rename(lambda x: x.strip(), axis='columns', inplace=True)

In [5]:
df.shape

(31696, 59)

In [6]:
df.head()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,label
0,8,810,0,1,0,16,7,1,0,4,...,0,0,0,0,0,0,0,0,0,919
1,12,122,0,1,0,7,2,1,0,4,...,0,1,0,0,0,0,0,0,0,1600
2,12,891,0,1,0,6,3,22,2,4,...,0,1,0,0,0,1,0,0,0,11700
3,9,1323,0,1,0,31,11,13,0,4,...,0,0,0,0,0,0,0,0,0,18000
4,8,261,0,1,0,8,3,4,0,4,...,0,1,0,0,0,0,0,0,0,5800


In [7]:
from synthesizers.metgan import *
metgan = MeTGANSynthesizer(embedding_dim=64,generator_dim=(256,256),discriminator_dim=(256,256),batch_size=500,discriminator_steps=5,verbose=True)

In [8]:
metgan.fit(df, discrete_columns, epochs=3)  # Epochs = 300 used in the actual experiments

Data_dim_p:  2530
Epoch 1, Loss G: -2.9062,Loss D: -0.6814
Epoch 2, Loss G: -0.3893,Loss D:  0.0767
Epoch 3, Loss G:  0.0647,Loss D: -0.3629


In [9]:
metgan.save(r'../models/model-news.pth')

In [10]:
syn_data = metgan.sample(39644)  # Size of complete News dataset

In [11]:
syn_data.head()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,label
0,10,648,0,1,0,5,3,0,0,4,...,0,0,0,-1,0,0,0,0,0,1574
1,10,1771,0,0,0,1,2,10,0,4,...,0,0,0,0,0,0,0,0,0,14921
2,9,399,0,1,0,21,3,0,0,4,...,0,0,0,0,0,0,0,0,0,1164
3,6,181,0,0,0,0,2,0,0,4,...,0,0,0,0,0,0,0,0,0,563
4,10,679,0,0,0,8,3,0,0,3,...,0,0,0,0,0,0,0,0,0,4627


In [12]:
syn_data.to_csv('../syn/metgan-news-3epochs.csv',index=False)